In [1]:
import ee
from ee import batch
ee.Initialize()

In [2]:
maca = ee.ImageCollection('IDAHO_EPSCOR/MACAv2_METDATA')
parks = ee.FeatureCollection('users/majo3748/conus_natl_parks')
parks = parks.filterMetadata('UNIT_CO', 'equals', 'WICA')
n_park = parks.size().getInfo()
park_list = parks.toList(n_park)

In [3]:
n_park

1

In [4]:
processes = list()

In [5]:
feat = ee.Feature(park_list.get(0))
unit_code = feat.get('UNIT_CO').getInfo()

## Get historical data

In [7]:
# get historical data
dates = [str(y) + '-01-01' for y in range(1950, 2007)]
for i in range(len(dates) - 1):
  maca_stack = maca.filterDate(dates[i], dates[i + 1]).toBands()
  means = maca_stack.reduceRegion(
    reducer = ee.Reducer.mean(), 
    geometry = feat.geometry(), 
    scale = 4000, 
    maxPixels = 1e16
  )
  out = batch.Export.table.toDrive(
    collection = ee.FeatureCollection([ee.Feature(None, means)]), 
    description = '_'.join([unit_code, dates[i][:4], 'historical']), 
    folder = 'maca-daily'
  )
  processes.append(batch.Task.start(out))

## Get future data, split by RCP scenario

In [9]:
# get historical data
dates = [str(y) + '-01-01' for y in range(2006, 2101)]
rcp = ['rcp85', 'rcp45']
for i in range(len(dates) - 1):
  for r in rcp:
    maca_stack = maca.filterDate(dates[i], dates[i + 1]).filterMetadata('scenario', 'equals', r).toBands()
    means = maca_stack.reduceRegion(
      reducer = ee.Reducer.mean(), 
      geometry = feat.geometry(), 
      scale = 4000, 
      maxPixels = 1e16
    )
    out = batch.Export.table.toDrive(
      collection = ee.FeatureCollection([ee.Feature(None, means)]), 
      description = '_'.join([unit_code, dates[i][:4], r]), 
      folder = 'maca-daily'
    )
    processes.append(batch.Task.start(out))